# GPT3 Finetuning


https://www.kaggle.com/datasets/hassanamin/atis-airlinetravelinformationsystem

In [39]:
!kaggle datasets download -d hassanamin/atis-airlinetravelinformationsystem

  0% 0.00/139k [00:00<?, ?B/s]
100% 139k/139k [00:00<00:00, 56.5MB/s]


In [40]:
!mkdir data

In [41]:
! unzip atis-airlinetravelinformationsystem.zip -d train

Archive:  atis-airlinetravelinformationsystem.zip
  inflating: train/atis_intents.csv  
  inflating: train/atis_intents_test.csv  
  inflating: train/atis_intents_train.csv  


In [42]:
import pandas as pd
import numpy as np

### Loading and cleaning data

In [43]:
data = pd.read_csv("train/atis_intents.csv",header=None)
data.head()

,0,1
0,atis_flight,i want to fly from boston at 838 am and arriv...
1,atis_flight,what flights are available from pittsburgh to...
2,atis_flight_time,what is the arrival time in san francisco for...
3,atis_airfare,cheapest airfare from tacoma to orlando
4,atis_airfare,round trip fares from pittsburgh to philadelp...


In [44]:
data.columns = ['intent','text']

In [45]:
data.head()

,intent,text
0,atis_flight,i want to fly from boston at 838 am and arriv...
1,atis_flight,what flights are available from pittsburgh to...
2,atis_flight_time,what is the arrival time in san francisco for...
3,atis_airfare,cheapest airfare from tacoma to orlando
4,atis_airfare,round trip fares from pittsburgh to philadelp...


In [46]:
data['intent'].unique()

array(['atis_flight', 'atis_flight_time', 'atis_airfare', 'atis_aircraft',
       'atis_ground_service', 'atis_airport', 'atis_airline',
       'atis_distance', 'atis_abbreviation', 'atis_ground_fare',
       'atis_quantity', 'atis_city', 'atis_flight_no', 'atis_capacity',
       'atis_flight#atis_airfare', 'atis_meal', 'atis_restriction',
       'atis_airline#atis_flight_no',
       'atis_ground_service#atis_ground_fare',
       'atis_airfare#atis_flight_time', 'atis_cheapest',
       'atis_aircraft#atis_flight#atis_flight_no'], dtype=object)

In [47]:
data['intent'].nunique()

22

Removing unnecessary characters and words from intent classes

In [48]:
data['intent'] = data['intent'].str.replace('#','_')
data['intent'].unique()

array(['atis_flight', 'atis_flight_time', 'atis_airfare', 'atis_aircraft',
       'atis_ground_service', 'atis_airport', 'atis_airline',
       'atis_distance', 'atis_abbreviation', 'atis_ground_fare',
       'atis_quantity', 'atis_city', 'atis_flight_no', 'atis_capacity',
       'atis_flight_atis_airfare', 'atis_meal', 'atis_restriction',
       'atis_airline_atis_flight_no',
       'atis_ground_service_atis_ground_fare',
       'atis_airfare_atis_flight_time', 'atis_cheapest',
       'atis_aircraft_atis_flight_atis_flight_no'], dtype=object)

In [49]:
data['intent'] = data['intent'].str.replace('atis_','')
data['intent'].unique()

array(['flight', 'flight_time', 'airfare', 'aircraft', 'ground_service',
       'airport', 'airline', 'distance', 'abbreviation', 'ground_fare',
       'quantity', 'city', 'flight_no', 'capacity', 'flight_airfare',
       'meal', 'restriction', 'airline_flight_no',
       'ground_service_ground_fare', 'airfare_flight_time', 'cheapest',
       'aircraft_flight_flight_no'], dtype=object)

In [50]:
data['intent'].value_counts()

flight                        3666
airfare                        423
ground_service                 255
airline                        157
abbreviation                   147
aircraft                        81
flight_time                     54
quantity                        51
flight_airfare                  21
airport                         20
distance                        20
city                            19
ground_fare                     18
capacity                        16
flight_no                       12
meal                             6
restriction                      6
airline_flight_no                2
ground_service_ground_fare       1
airfare_flight_time              1
cheapest                         1
aircraft_flight_flight_no        1
Name: intent, dtype: int64

Reducing our data to only five intent classes with 40 samples each. Not recommended as samples of each class should be atleast 100 but this will save training cost (as this notebook is only for educational purposes)

In [51]:
labels = ['flight','ground_service','airfare','abbreviation','flight_time']
data = data[data["intent"].isin(labels)]
data['intent'].value_counts()

flight            3666
airfare            423
ground_service     255
abbreviation       147
flight_time         54
Name: intent, dtype: int64

In [ ]:
sample_data = data.groupby('intent').apply(lambda x: x.sample(n=40)).reset_index(drop = True)

In [53]:
sample_data['intent'].value_counts()

abbreviation      40
airfare           40
flight            40
flight_time       40
ground_service    40
Name: intent, dtype: int64

In [54]:
sample_data.to_csv("sample_data.csv",index=False)

In [ ]:
sample_data = sample_data[['text','intent']]
sample_data.head()

In [56]:
#Leave no extra space on left or right - sanity check
sample_data['text'] = sample_data['text'].str.strip()
sample_data['intent'] = sample_data['intent'].str.strip()

Adding end tokens to prompt and response

In [57]:
sample_data['text'] = sample_data['text'] + "\n\nIntent:\n\n"
# sample_data['text'] = "Classify text into on the intent: flight, ground_service, airline, aircraft, flight_time. Text: "+sample_data['text'] + "\n\nIntent:\n\n"
sample_data['intent'] = " "+sample_data['intent'] + " END"
sample_data.head()

,text,intent
0,what does the fare code y mean\n\nIntent:\n\n,abbreviation END
1,could you tell me what the abbreviation us sta...,abbreviation END
2,what is fare code m\n\nIntent:\n\n,abbreviation END
3,what is sa\n\nIntent:\n\n,abbreviation END
4,what is the ap57 restriction\n\nIntent:\n\n,abbreviation END


In [58]:
print(sample_data['text'][0])

what does the fare code y mean

Intent:




In [59]:
print(sample_data['intent'][0])

 abbreviation END


To prepare data for training, we change the columns name to prompt and completion

In [61]:
sample_data.columns = ['prompt','completion']
sample_data.head()

,prompt,completion
0,what does the fare code y mean\n\nIntent:\n\n,abbreviation END
1,could you tell me what the abbreviation us sta...,abbreviation END
2,what is fare code m\n\nIntent:\n\n,abbreviation END
3,what is sa\n\nIntent:\n\n,abbreviation END
4,what is the ap57 restriction\n\nIntent:\n\n,abbreviation END


Converting our pandas dataframe to jsonl file (required format for gpt3 finetuning)

In [62]:
sample_data.to_json("intent_sample.jsonl", orient='records', lines=True)

# Finetuning GPT3

Installing openAI

In [63]:
!pip install --upgrade openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai: filename=openai-0.26.5-py3-none-any.whl size=67620 sha256=e2dc811b0fb4f777e181ac0f136180725089a4c58d40fa264da958d5d3d06b25
  Stored in directory: /root/.cache/pip/wheels/a7/47/99/8273a59fbd59c303e8ff175416d5c1c9c03a2e83ebf7525a99
Successfully built openai


Uploading training data

In [64]:
!openai tools fine_tunes.prepare_data -f intent_sample.jsonl


Analyzing...

- Your file contains 200 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 9 duplicated prompt-completion sets. These are rows: [16, 20, 26, 27, 28, 32, 35, 36, 37]
- All prompts end with suffix `\n\nIntent:\n\n`. This suffix seems very long. Consider replacing with a shorter suffix, such as `\n\n###\n\n`

Based on the analysis we will perform the following actions:
- [Recommended] Remove 9 duplicate rows [Y/n]: y
- [Recommended] Would you like to split into training and validation set? [Y/n]: y


Your data will be written to a new JSONL file. Proceed [Y/n]: y

Wrote modified files to `intent_sample_prepared_train.jsonl` and `intent_sample_prepared_valid.jsonl`
Feel free to t

Loading API key

In [65]:
import os
os.environ['OPENAI_API_KEY'] = "your openai api key"

Starting finetuning

In [ ]:
!openai api fine_tunes.create -t "intent_sample_prepared_train.jsonl" -v "intent_sample_prepared_valid.jsonl" -m 'davinci'

In [70]:
!openai api fine_tunes.follow -i ft-ujcjyJIzsG9rXuLfobVc6yaV

[2023-02-14 15:47:03] Created fine-tune: ft-ujcjyJIzsG9rXuLfobVc6yaV
[2023-02-14 15:53:26] Fine-tune costs $0.40
[2023-02-14 15:53:27] Fine-tune enqueued
[2023-02-14 16:04:13] Fine-tune is in the queue. Queue number: 31
[2023-02-14 16:10:21] Fine-tune is in the queue. Queue number: 30
[2023-02-14 16:11:28] Fine-tune is in the queue. Queue number: 29
[2023-02-14 16:12:47] Fine-tune is in the queue. Queue number: 28
[2023-02-14 16:13:10] Fine-tune is in the queue. Queue number: 27
[2023-02-14 16:15:15] Fine-tune is in the queue. Queue number: 26
[2023-02-14 16:17:22] Fine-tune is in the queue. Queue number: 25
[2023-02-14 16:17:25] Fine-tune is in the queue. Queue number: 24
[2023-02-14 16:20:24] Fine-tune is in the queue. Queue number: 23
[2023-02-14 16:20:26] Fine-tune is in the queue. Queue number: 22
[2023-02-14 16:21:09] Fine-tune is in the queue. Queue number: 21
[2023-02-14 16:22:23] Fine-tune is in the queue. Queue number: 20
[2023-02-14 16:22:48] Fine-tune is in the queue. Queue

Testing finetuned model on different prompts

In [71]:
prompt = "Do we have london flight on Monday\n\nIntent:\n\n"

In [74]:
import openai
openai.api_key ='your openai api key'
response = openai.Completion.create(
  model="davinci:ft-personal-2023-02-14-16-48-51",
  prompt=prompt,
  max_tokens=5,
  temperature=0,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=[" END"]
)
print(response['choices'][0]['text'])

 flight


In [75]:
prompt = "what is the ap57 restriction\n\nIntent:\n\n"

In [76]:
import openai
openai.api_key ='your openai api key'
response = openai.Completion.create(
  model="davinci:ft-personal-2023-02-14-16-48-51",
  prompt=prompt,
  max_tokens=5,
  temperature=0,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=[" END"]
)
print(response['choices'][0]['text'])

 abbreviation


In [77]:
prompt = "show me ground transportation in baltimore\n\nIntent:\n\n"

In [78]:
import openai
openai.api_key ='your openai api key'
response = openai.Completion.create(
  model="davinci:ft-personal-2023-02-14-16-48-51",
  prompt=prompt,
  max_tokens=5,
  temperature=0,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=[" END"]
)
print(response['choices'][0]['text'])

 ground_service


As can be seen, our model is successfully finetuned.